# Import Tweets

In [91]:
# Import all libraries 
import pandas as pd
import numpy as np 
import datetime

In [92]:
trump_tweets = pd.read_csv("trump_tweets_cleaned.csv")
trump_tweets['date'] = pd.to_datetime(trump_tweets['date'])
trump_tweets.rename(columns={"date": "tweet_datetime"}, inplace=True)

In [93]:
trump_tweets

,id,cleaned_text,favorites,retweets,tweet_datetime,date_new,date_part,time_part,hour,year,month
0,9.384230e+17,MAKE AMERICA GREAT AGAIN!,157963,37189,2017-06-12 15:00:00,2017-06-12 23:00:00,2017-06-12,23:00:00,23,2017,6
1,9.391890e+17,MAKE AMERICA GREAT AGAIN!,56596,11433,2017-08-12 17:46:00,2017-08-13 01:46:00,2017-08-13,01:46:00,1,2017,8
2,9.353400e+17,Thank you Rand!,42793,9125,2017-11-28 02:50:00,2017-11-28 10:50:00,2017-11-28,10:50:00,10,2017,11
3,9.253890e+17,Thank you @LuisRiveraMarin!,23521,4574,2017-10-31 15:48:00,2017-10-31 23:48:00,2017-10-31,23:48:00,23,2017,10
4,8.997980e+17,"Join me live from Fort Myer in Arlington, Virg...",36009,4891,2017-08-22 01:00:00,2017-08-22 09:00:00,2017-08-22,09:00:00,9,2017,8
...,...,...,...,...,...,...,...,...,...,...,...
5083,9.862190e+17,"Employment is up, Taxes are DOWN. Enjoy! I am ...",106514,19661,2018-04-17 12:24:00,2018-04-17 20:24:00,2018-04-17,20:24:00,20,2018,4
5084,9.791090e+17,I am pleased to announce that I intend to nomi...,66173,13399,2018-03-28 21:31:00,2018-03-29 05:31:00,2018-03-29,05:31:00,5,2018,3
5085,9.597990e+17,Rasmussen just announced that my approval rati...,132165,30235,2018-03-02 14:40:00,2018-03-02 22:40:00,2018-03-02,22:40:00,22,2018,3
5086,9.576040e+17,Somebody please inform Jay-Z that because of m...,202727,49876,2018-01-28 13:18:00,2018-01-28 21:18:00,2018-01-28,21:18:00,21,2018,1


# Import Stock Prices

In [100]:
stock_price = pd.read_csv("stock_price_cleaned.csv")

In [101]:
stock_price.rename(columns={"DATE_TIME_M": "datetime", "PRICE": "price"}, inplace=True)

In [102]:
stock_price

,DATE,TIME_M,price
0,20170120,04:00:00,226.50
1,20170120,04:04:00,226.49
2,20170120,04:05:00,226.52
3,20170120,04:08:00,226.54
4,20170120,04:10:00,226.52
...,...,...,...
381308,20181231,19:56:00,250.06
381309,20181231,19:57:00,250.07
381310,20181231,19:58:00,250.10
381311,20181231,19:59:00,250.07


# Get End of Day (8pm) Stock Prices

In [98]:
end_of_day_prices = stock_price[stock_price['datetime']]

KeyError: 'datetime'

# Get Stock Price Difference X mins after

In [71]:
# Change this variable for different X min differences
x_mins = 60

In [72]:
stock_price['datetime']= pd.to_datetime(stock_price['datetime'])
stock_price[f"datetime_{x_mins}mins_before"] = stock_price["datetime"] - pd.Timedelta(f"{x_mins} min")

In [73]:
stock_price_merged = stock_price.merge(stock_price, how='inner', left_on=[f"datetime_{x_mins}mins_before"], right_on=["datetime"])
stock_price_merged.drop(columns=["datetime_y", f"datetime_{x_mins}mins_before_y"], inplace=True)
stock_price_merged.rename(columns={"datetime_x": f"datetime_{x_mins}mins_after", "price_x": f"price_{x_mins}mins_after", f"datetime_{x_mins}mins_before_x": "datetime_now", "price_y": "price_now"}, inplace=True)

In [74]:
stock_price

,datetime,price,datetime_60mins_before
0,2017-01-20 04:00:00,226.5000,2017-01-20 03:00:00
1,2017-01-20 04:01:00,226.4975,2017-01-20 03:01:00
2,2017-01-20 04:02:00,226.4950,2017-01-20 03:02:00
3,2017-01-20 04:03:00,226.4925,2017-01-20 03:03:00
4,2017-01-20 04:04:00,226.4900,2017-01-20 03:04:00
...,...,...,...
1023356,2018-12-31 19:56:00,250.0600,2018-12-31 18:56:00
1023357,2018-12-31 19:57:00,250.0700,2018-12-31 18:57:00
1023358,2018-12-31 19:58:00,250.1000,2018-12-31 18:58:00
1023359,2018-12-31 19:59:00,250.0700,2018-12-31 18:59:00


In [75]:
stock_price_merged['price_diff_abs'] = stock_price_merged[f'price_{x_mins}mins_after'] - stock_price_merged['price_now']

stock_price_merged['price_diff_perc'] = (stock_price_merged[f'price_{x_mins}mins_after'] - stock_price_merged['price_now']) / stock_price_merged['price_now']

In [76]:
stock_price_merged

,datetime_60mins_after,price_60mins_after,datetime_now,price_now,price_diff_abs,price_diff_perc
0,2017-01-20 05:00:00,226.563333,2017-01-20 04:00:00,226.5000,0.063333,0.000280
1,2017-01-20 05:01:00,226.556667,2017-01-20 04:01:00,226.4975,0.059167,0.000261
2,2017-01-20 05:02:00,226.550000,2017-01-20 04:02:00,226.4950,0.055000,0.000243
3,2017-01-20 05:03:00,226.560000,2017-01-20 04:03:00,226.4925,0.067500,0.000298
4,2017-01-20 05:04:00,226.561667,2017-01-20 04:04:00,226.4900,0.071667,0.000316
...,...,...,...,...,...,...
1023296,2018-12-31 19:56:00,250.060000,2018-12-31 18:56:00,249.9900,0.070000,0.000280
1023297,2018-12-31 19:57:00,250.070000,2018-12-31 18:57:00,249.9800,0.090000,0.000360
1023298,2018-12-31 19:58:00,250.100000,2018-12-31 18:58:00,250.0000,0.100000,0.000400
1023299,2018-12-31 19:59:00,250.070000,2018-12-31 18:59:00,250.0000,0.070000,0.000280


# Ge

# Merge Tweets and Stock Prices

In [77]:
merged = trump_tweets.merge(stock_price_merged, how='inner', left_on=['tweet_datetime'], right_on=["datetime_now"])
merged.drop(columns=["datetime_now", "price_now"], inplace=True)

In [78]:
merged.shape

(5087, 15)

In [79]:
merged

,id,cleaned_text,favorites,retweets,tweet_datetime,date_new,date_part,time_part,hour,year,month,datetime_60mins_after,price_60mins_after,price_diff_abs,price_diff_perc
0,9.384230e+17,MAKE AMERICA GREAT AGAIN!,157963,37189,2017-06-12 15:00:00,2017-06-12 23:00:00,2017-06-12,23:00:00,23,2017,6,2017-06-12 16:00:00,243.330000,0.365000,0.001502
1,9.391890e+17,MAKE AMERICA GREAT AGAIN!,56596,11433,2017-08-12 17:46:00,2017-08-13 01:46:00,2017-08-13,01:46:00,1,2017,8,2017-08-12 18:46:00,244.803000,0.030000,0.000123
2,9.353400e+17,Thank you Rand!,42793,9125,2017-11-28 02:50:00,2017-11-28 10:50:00,2017-11-28,10:50:00,10,2017,11,2017-11-28 03:50:00,260.347500,0.015000,0.000058
3,9.253890e+17,Thank you @LuisRiveraMarin!,23521,4574,2017-10-31 15:48:00,2017-10-31 23:48:00,2017-10-31,23:48:00,23,2017,10,2017-10-31 16:48:00,257.163333,-0.001667,-0.000006
4,8.997980e+17,"Join me live from Fort Myer in Arlington, Virg...",36009,4891,2017-08-22 01:00:00,2017-08-22 09:00:00,2017-08-22,09:00:00,9,2017,8,2017-08-22 02:00:00,243.357500,0.076250,0.000313
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5082,9.862190e+17,"Employment is up, Taxes are DOWN. Enjoy! I am ...",106514,19661,2018-04-17 12:24:00,2018-04-17 20:24:00,2018-04-17,20:24:00,20,2018,4,2018-04-17 13:24:00,269.910000,-0.340000,-0.001258
5083,9.791090e+17,I am pleased to announce that I intend to nomi...,66173,13399,2018-03-28 21:31:00,2018-03-29 05:31:00,2018-03-29,05:31:00,5,2018,3,2018-03-28 22:31:00,260.081667,0.100000,0.000385
5084,9.597990e+17,Rasmussen just announced that my approval rati...,132165,30235,2018-03-02 14:40:00,2018-03-02 22:40:00,2018-03-02,22:40:00,22,2018,3,2018-03-02 15:40:00,268.930000,0.890000,0.003320
5085,9.576040e+17,Somebody please inform Jay-Z that because of m...,202727,49876,2018-01-28 13:18:00,2018-01-28 21:18:00,2018-01-28,21:18:00,21,2018,1,2018-01-28 14:18:00,286.421375,-0.003750,-0.000013


In [80]:
merged['price_diff_perc_magnitude'] = merged['price_diff_perc'].abs()

In [81]:
merged.head()

,id,cleaned_text,favorites,retweets,tweet_datetime,date_new,date_part,time_part,hour,year,month,datetime_60mins_after,price_60mins_after,price_diff_abs,price_diff_perc,price_diff_perc_magnitude
0,9.384230e+17,MAKE AMERICA GREAT AGAIN!,157963,37189,2017-06-12 15:00:00,2017-06-12 23:00:00,2017-06-12,23:00:00,23,2017,6,2017-06-12 16:00:00,243.330000,0.365000,0.001502,0.001502
1,9.391890e+17,MAKE AMERICA GREAT AGAIN!,56596,11433,2017-08-12 17:46:00,2017-08-13 01:46:00,2017-08-13,01:46:00,1,2017,8,2017-08-12 18:46:00,244.803000,0.030000,0.000123,0.000123
2,9.353400e+17,Thank you Rand!,42793,9125,2017-11-28 02:50:00,2017-11-28 10:50:00,2017-11-28,10:50:00,10,2017,11,2017-11-28 03:50:00,260.347500,0.015000,0.000058,0.000058
3,9.253890e+17,Thank you @LuisRiveraMarin!,23521,4574,2017-10-31 15:48:00,2017-10-31 23:48:00,2017-10-31,23:48:00,23,2017,10,2017-10-31 16:48:00,257.163333,-0.001667,-0.000006,0.000006
4,8.997980e+17,"Join me live from Fort Myer in Arlington, Virg...",36009,4891,2017-08-22 01:00:00,2017-08-22 09:00:00,2017-08-22,09:00:00,9,2017,8,2017-08-22 02:00:00,243.357500,0.076250,0.000313,0.000313


In [82]:
merged_sorted = merged.sort_values(by=['price_diff_perc_magnitude'], ascending=False)

In [87]:
merged_sorted.head(50)

,id,cleaned_text,favorites,retweets,tweet_datetime,date_new,date_part,time_part,hour,year,month,datetime_60mins_after,price_60mins_after,price_diff_abs,price_diff_perc,price_diff_perc_magnitude
3234,1.036240e+18,Tiger Woods showed great class in the way he a...,107438,21105,2018-02-09 13:28:00,2018-02-09 21:28:00,2018-02-09,21:28:00,21,2018,2,2018-02-09 14:28:00,259.600000,5.930000,0.023377,0.023377
2316,1.075400e+18,"We have defeated ISIS in Syria, my only reason...",100443,20265,2018-12-19 14:29:00,2018-12-19 22:29:00,2018-12-19,22:29:00,22,2018,12,2018-12-19 15:29:00,251.110000,-5.270000,-0.020555,0.020555
3233,1.036250e+18,....The fact is that African/American unemploy...,117688,26036,2018-02-09 13:37:00,2018-02-09 21:37:00,2018-02-09,21:37:00,21,2018,2,2018-02-09 14:37:00,258.310000,4.799000,0.018930,0.018930
4116,1.002880e+18,“We ran out of words to describe how good the ...,63206,12913,2018-02-06 11:40:00,2018-02-06 19:40:00,2018-02-06,19:40:00,19,2018,2,2018-02-06 12:40:00,266.390000,4.030000,0.015361,0.015361
2735,1.056920e+18,Many Gang Members and some very bad people are...,122928,32961,2018-10-29 14:41:00,2018-10-29 22:41:00,2018-10-29,22:41:00,22,2018,10,2018-10-29 15:41:00,261.230000,-3.930000,-0.014821,0.014821
2315,1.075400e+18,The Trump Foundation has done great work and g...,56837,13069,2018-12-19 14:44:00,2018-12-19 22:44:00,2018-12-19,22:44:00,22,2018,12,2018-12-19 15:44:00,250.726800,-3.763200,-0.014787,0.014787
2223,1.078660e+18,Thank you to Sean Parnell for the nice comment...,100626,17587,2018-12-28 14:51:00,2018-12-28 22:51:00,2018-12-28,22:51:00,22,2018,12,2018-12-28 15:51:00,247.370000,-3.370000,-0.013440,0.013440
2286,1.076130e+18,There has never been a president who has been ...,67378,13875,2018-12-21 14:41:00,2018-12-21 22:41:00,2018-12-21,22:41:00,22,2018,12,2018-12-21 15:41:00,240.420000,-2.810000,-0.011553,0.011553
3223,1.036990e+18,"NBC FAKE NEWS, which is under intense scrutiny...",88482,20438,2018-04-09 14:58:00,2018-04-09 22:58:00,2018-04-09,22:58:00,22,2018,4,2018-04-09 15:58:00,260.775000,-3.025000,-0.011467,0.011467
2284,1.076130e+18,The Democrats now own the shutdown!,111704,23616,2018-12-21 15:07:00,2018-12-21 23:07:00,2018-12-21,23:07:00,23,2018,12,2018-12-21 16:07:00,240.210000,-2.760000,-0.011359,0.011359
